In [2]:
# -----------------------------
# Helper input functions
# -----------------------------

def ask_choice(question, options):
    while True:
        print("\n" + question)
        for i, opt in enumerate(options, 1):
            print(f"{i}. {opt}")
        choice = input("Enter number: ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(options):
            return options[int(choice) - 1]
        print("Invalid input. Please try again.")


def ask_yes_some_no(question):
    options = ["Yes", "Some", "No"]
    return ask_choice(question, options)


def ask_yes_no(question):
    options = ["Yes", "No"]
    return ask_choice(question, options)


def ask_number(question, min_value=0):
    while True:
        val = input(f"\n{question}: ").strip()
        try:
            num = float(val)
            if num >= min_value:
                return num
        except ValueError:
            pass
        print("Please enter a valid number.")


def ask_text(question):
    return input(f"\n{question}: ").strip()


# -----------------------------
# Scoring helpers
# -----------------------------

YES_SOME_NO_SCORE = {
    "Yes": 1.0,
    "Some": 0.5,
    "No": 0.0
}

INTENSITY_SCORE = {
    "Light": 0.3,
    "Moderate": 0.6,
    "Heavy": 1.0
}

TIMING_RISK_SCORE = {
    "Daytime only": 0.0,
    "Early morning / night": 0.4
}


def cap_score(value):
    return max(0, min(100, int(round(value))))


# -----------------------------
# Main program
# -----------------------------

def main():
    print("\n=== Employer Job Characterisation Tool ===")

    # A. Job Identity
    job_category = ask_choice(
        "Select job category",
        ["Agriculture", "Construction", "Factory / Workshop", "General Labour"]
    )

    description = ask_text("Short description of work")
    workers_required = int(ask_number("Number of workers required", 1))
    duration_days = int(ask_number("Expected duration (in days)", 1))

    # B. Skill Demand Profile
    physical_strength = ask_yes_some_no("Physical strength required?")
    tool_use = ask_yes_some_no("Tool or machinery use?")
    precision = ask_yes_some_no("Precision or careful work?")
    instructions = ask_yes_some_no("Ability to follow instructions?")
    independence = ask_yes_some_no("Independent work without supervision?")

    # C. Risk & Intensity Profile
    intensity = ask_choice(
        "Physical intensity level",
        ["Light", "Moderate", "Heavy"]
    )

    sharp_tools = ask_yes_no("Safety risk: sharp tools?")
    heavy_machinery = ask_yes_no("Safety risk: heavy machinery?")
    work_height = ask_yes_no("Safety risk: work at height?")
    extreme_weather = ask_yes_no("Safety risk: extreme heat or weather?")

    timing = ask_choice(
        "Work timing",
        ["Daytime only", "Early morning / night"]
    )

    # D. Job Conditions
    daily_wage = ask_number("Daily wage (amount)", 0)
    payment_frequency = ask_choice(
        "Payment frequency",
        ["Daily", "Weekly", "End of job"]
    )

    food = ask_yes_no("Food provided?")
    transport = ask_yes_no("Transport provided?")
    accommodation = ask_yes_no("Accommodation provided?")
    distance_km = ask_number("Distance from nearest village (km)", 0)

    # E. Flexibility & Inclusion Flags
    allows_unskilled = ask_yes_no("Can unskilled workers do this job?")
    guidance = ask_yes_no("Is on-site guidance or supervision provided?")
    women_safe = ask_yes_no("Can women safely work at this site?")
    older_workers = ask_yes_no("Can older workers (45+) do this job?")

    # -----------------------------
    # Scoring Logic
    # -----------------------------

    # Job Complexity Score
    complexity_raw = (
        YES_SOME_NO_SCORE[physical_strength] * 15 +
        YES_SOME_NO_SCORE[tool_use] * 25 +
        YES_SOME_NO_SCORE[precision] * 25 +
        YES_SOME_NO_SCORE[instructions] * 15 +
        YES_SOME_NO_SCORE[independence] * 20
    )
    complexity_score = cap_score(complexity_raw)

    # Job Risk Score
    risk_raw = (
        INTENSITY_SCORE[intensity] * 40 +
        (sharp_tools == "Yes") * 15 +
        (heavy_machinery == "Yes") * 20 +
        (work_height == "Yes") * 15 +
        (extreme_weather == "Yes") * 10 +
        TIMING_RISK_SCORE[timing] * 20
    )
    risk_score = cap_score(risk_raw)

    # Job Accessibility Score
    accessibility_raw = 0
    accessibility_raw += 30 if allows_unskilled == "Yes" else 0
    accessibility_raw += 20 if guidance == "Yes" else 0
    accessibility_raw += max(0, 20 - risk_score * 0.2)
    accessibility_raw += max(0, 15 - duration_days)
    accessibility_raw += max(0, 15 - distance_km)

    accessibility_score = cap_score(accessibility_raw)

    # -----------------------------
    # Output Report
    # -----------------------------

    print("\n" + "=" * 50)
    print("JOB PROFILE REPORT")
    print("=" * 50)

    print(f"\nJob Category: {job_category}")
    print(f"Description: {description}")
    print(f"Workers Required: {workers_required}")
    print(f"Expected Duration: {duration_days} days")

    print("\n--- Job Complexity Score:", complexity_score, "/ 100 ---")
    print("- Physical strength:", physical_strength)
    print("- Tool or machinery use:", tool_use)
    print("- Precision required:", precision)
    print("- Instruction following:", instructions)
    print("- Independent work:", independence)

    print("\n--- Job Risk Score:", risk_score, "/ 100 ---")
    print("- Physical intensity:", intensity)
    print("- Sharp tools:", sharp_tools)
    print("- Heavy machinery:", heavy_machinery)
    print("- Work at height:", work_height)
    print("- Extreme weather:", extreme_weather)
    print("- Work timing:", timing)

    print("\n--- Job Accessibility Score:", accessibility_score, "/ 100 ---")
    print("- Allows unskilled workers:", allows_unskilled)
    print("- Guidance provided:", guidance)
    print("- Distance from village:", distance_km, "km")
    print("- Duration:", duration_days, "days")

    print("\n--- Plain-Language Summary ---")
    summary_parts = []
    if intensity in ["Moderate", "Heavy"]:
        summary_parts.append("requires physical effort")
    if tool_use in ["Yes", "Some"]:
        summary_parts.append("involves tool or equipment use")
    if guidance == "Yes":
        summary_parts.append("includes on-site guidance")
    if allows_unskilled == "Yes":
        summary_parts.append("can be done by unskilled workers")
    if risk_score > 60:
        summary_parts.append("has notable safety risks")

    if summary_parts:
        print("This job " + ", ".join(summary_parts) + ".")
    else:
        print("This job has simple requirements and low physical risk.")

    print("=" * 50)
    print("End of Report")


if __name__ == "__main__":
    main()



=== Employer Job Characterisation Tool ===

Select job category
1. Agriculture
2. Construction
3. Factory / Workshop
4. General Labour


Enter number:  1

Short description of work:  Need Some people to harvest corn

Number of workers required:  10

Expected duration (in days):  7



Physical strength required?
1. Yes
2. Some
3. No


Enter number:  2



Tool or machinery use?
1. Yes
2. Some
3. No


...


Can women safely work at this site?
1. Yes
2. No


Enter number:  1



Can older workers (45+) do this job?
1. Yes
2. No


Enter number:  2



JOB PROFILE REPORT

Job Category: Agriculture
Description: Need Some people to harvest corn
Workers Required: 10
Expected Duration: 7 days

--- Job Complexity Score: 60 / 100 ---
- Physical strength: Some
- Tool or machinery use: Some
- Precision required: Some
- Instruction following: Some
- Independent work: Yes

--- Job Risk Score: 85 / 100 ---
- Physical intensity: Heavy
- Sharp tools: Yes
- Heavy machinery: Yes
- Work at height: No
- Extreme weather: Yes
- Work timing: Daytime only

--- Job Accessibility Score: 61 / 100 ---
- Allows unskilled workers: Yes
- Guidance provided: Yes
- Distance from village: 20.0 km
- Duration: 7 days

--- Plain-Language Summary ---
This job requires physical effort, involves tool or equipment use, includes on-site guidance, can be done by unskilled workers, has notable safety risks.
End of Report
